In [1]:
%reset -f
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [2]:
# Load an image file and set the resolution
photos = np.load('Cats_Dogs_photos.npy')
labels = np.load('Cats_Dogs_labels.npy')

X_train = photos[:1600,:,:,:]
X_test = photos[1600:,:,:,:]
y_train = labels[:1600]
y_test = labels[1600:]

# preprocessing input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

# load the convolutional base model and set layers as not trainable
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
base_model = MobileNetV2(include_top = False, input_shape = (192,192,3))

for layer in base_model.layers:
    layer.trainable = False

In [3]:
# add new classifier layers
x = Flatten()(base_model.layers[-1].output)
x = Dense(128, activation = 'relu', kernel_initializer = 'he_uniform')(x)
output = Dense(1, activation = 'sigmoid')(x)

# define new model, compile, fit
model = Model(inputs = base_model.inputs, outputs = output)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 4, batch_size = 64, validation_data =(X_test,y_test))

Epoch 1/4
25/25 [==============================] - 23s 823ms/step - loss: 1.0840 - accuracy: 0.8489 - val_loss: 0.1377 - val_accuracy: 0.9825
Epoch 2/4
25/25 [==============================] - 19s 777ms/step - loss: 0.0252 - accuracy: 0.9935 - val_loss: 0.1144 - val_accuracy: 0.9825
Epoch 3/4
25/25 [==============================] - 19s 790ms/step - loss: 0.0017 - accuracy: 0.9996 - val_loss: 0.1495 - val_accuracy: 0.9800
Epoch 4/4
25/25 [==============================] - 20s 791ms/step - loss: 0.0087 - accuracy: 0.9986 - val_loss: 0.1360 - val_accuracy: 0.9875


In [26]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow
image = load_img('linhmieu.jpg', target_size = (192,192,3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

from tensorflow.keras.applications.mobilenet_v2 import decode_predictions
pred = model.predict(image)

# pred = 0 => it is a dog
# pred = 1 => it is a cat
pred

array([[0.9999989]], dtype=float32)

In [34]:
from tensorflow.keras import Input
from tensorflow.keras import models
from tensorflow.keras import layers

x0 = Input(shape = (192,192,3))

x1 = layers.Conv2D(32,(3,3), strides = (1,1),activation  = 'relu', input_shape = (192,192,3))(x0)
x2 = layers.MaxPooling2D((2,2))(x1)

x3 = layers.Conv2D(64,(3,3), strides = (1,1),activation = 'relu')(x2)
x4 = layers.MaxPooling2D((2,2))(x3)

x5 = layers.Flatten()(x4)

x6 = layers.Dense(64, activation = 'relu')(x5)
x7 = layers.Dense(1, activation = 'sigmoid')(x6)

model1 = Model(inputs = x0, outputs = x7)

model1.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])
model1.fit(X_train,y_train,epochs = 4, batch_size = 64,
                 validation_data = (X_test,y_test))

Epoch 1/4
25/25 [==============================] - 14s 550ms/step - loss: 1.3554 - accuracy: 0.5370 - val_loss: 0.6388 - val_accuracy: 0.5925
Epoch 2/4
25/25 [==============================] - 14s 542ms/step - loss: 0.6049 - accuracy: 0.6454 - val_loss: 0.6240 - val_accuracy: 0.6350
Epoch 3/4
25/25 [==============================] - 13s 535ms/step - loss: 0.5340 - accuracy: 0.7320 - val_loss: 0.6101 - val_accuracy: 0.6600
Epoch 4/4
25/25 [==============================] - 13s 519ms/step - loss: 0.4265 - accuracy: 0.8206 - val_loss: 0.6026 - val_accuracy: 0.6725


In [35]:
pred1 = model1.predict(image)

# pred = 0 => it is a dog
# pred = 1 => it is a cat
pred1

array([[0.74840367]], dtype=float32)

On comparare notre résultat et le résultat par un réseau pré-entrainé.
On remarque le niveau de précision de pré-entrainé apport une taux de 99.99% pourtant notre réseau dont 74,8%.